<a href="https://colab.research.google.com/github/myrondza10/employee_scheduling_optimization/blob/main/employee_scheduling_optimizaiton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install ortools

In [3]:
from ortools.sat.python import cp_model
import pandas as pd

# This program tries to find an optimal assignment of employees to shifts
# (3 shifts per day, for 7 days), subject to some constraints (see below).
# Each employee can request to be assigned to specific shifts.
# The optimal assignment maximizes the number of fulfilled shift requests.
num_employees = 5
num_shifts = 3
num_days = 7
all_employees = range(num_employees)
all_shifts = range(num_shifts)
all_days = range(num_days)
shift_requests = [
    [[0, 0, 1], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 1], [0, 1, 0], [0, 0, 1]],
    [[0, 0, 0], [0, 0, 0], [0, 1, 0], [0, 1, 0], [1, 0, 0], [0, 0, 0], [0, 0, 1]],
    [[0, 1, 0], [0, 1, 0], [0, 0, 0], [1, 0, 0], [0, 0, 0], [0, 1, 0], [0, 0, 0]],
    [[0, 0, 1], [0, 0, 0], [1, 0, 0], [0, 1, 0], [0, 0, 0], [1, 0, 0], [0, 0, 0]],
    [[0, 0, 0], [0, 0, 1], [0, 1, 0], [0, 0, 0], [1, 0, 0], [0, 1, 0], [0, 0, 0]]
]
requested_df = pd.DataFrame(shift_requests)

In [4]:
requested_df

,0,1,2,3,4,5,6
0,"[0, 0, 1]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 1]","[0, 1, 0]","[0, 0, 1]"
1,"[0, 0, 0]","[0, 0, 0]","[0, 1, 0]","[0, 1, 0]","[1, 0, 0]","[0, 0, 0]","[0, 0, 1]"
2,"[0, 1, 0]","[0, 1, 0]","[0, 0, 0]","[1, 0, 0]","[0, 0, 0]","[0, 1, 0]","[0, 0, 0]"
3,"[0, 0, 1]","[0, 0, 0]","[1, 0, 0]","[0, 1, 0]","[0, 0, 0]","[1, 0, 0]","[0, 0, 0]"
4,"[0, 0, 0]","[0, 0, 1]","[0, 1, 0]","[0, 0, 0]","[1, 0, 0]","[0, 1, 0]","[0, 0, 0]"


In [5]:
# Creates the model.
model = cp_model.CpModel()

# Creates shift variables.
# shifts[(n, d, s)]: employees 'n' works shift 's' on day 'd'.
shifts = {}
for n in all_employees:
    for d in all_days:
        for s in all_shifts:
            shifts[(n, d,
                    s)] = model.NewBoolVar('shift_n%id%is%i' % (n, d, s))

# Each shift is assigned to exactly one employee in .
for d in all_days:
    for s in all_shifts:
        model.Add(sum(shifts[(n, d, s)] for n in all_employees) == 1)

# Each employee works at most one shift per day.
for n in all_employees:
    for d in all_days:
        model.Add(sum(shifts[(n, d, s)] for s in all_shifts) <= 1)

In [7]:
# Try to distribute the shifts evenly, so that each employee works
# min_shifts_per_emp shifts. If this is not possible, because the total
# number of shifts is not divisible by the number of employees, some employees will
# be assigned one more shift.

min_shifts_per_emp = (num_shifts * num_days) // num_employees
if num_shifts * num_days % num_employees == 0:
    max_shifts_per_emp = min_shifts_per_emp
else:
    max_shifts_per_emp = min_shifts_per_emp + 1
for n in all_employees:
    num_shifts_worked = 0
    for d in all_days:
        for s in all_shifts:
            num_shifts_worked += shifts[(n, d, s)]
    model.Add(min_shifts_per_emp <= num_shifts_worked)
    model.Add(num_shifts_worked <= max_shifts_per_emp)

In [8]:
# pylint: disable=g-complex-comprehension
model.Maximize(
    sum(shift_requests[n][d][s] * shifts[(n, d, s)] for n in all_employees
        for d in all_days for s in all_shifts))
# Creates the solver and solve.
solver = cp_model.CpSolver()
solver.Solve(model)

results = {d:{} for d in all_days}
for d in all_days:
    print('Day', d)
    for n in all_employees:
        for s in all_shifts:
            if solver.Value(shifts[(n, d, s)]) == 1:
                results[d][s] = n
                if shift_requests[n][d][s] == 1:
                    print('Employee', n, 'Works shift', s, '(requested).')
                else:
                    print('Employee', n, 'Works shift', s, '(not requested).')
    print()

Day 0
Employee 0 Works shift 2 (requested).
Employee 1 Works shift 0 (not requested).
Employee 2 Works shift 1 (requested).

Day 1
Employee 0 Works shift 0 (not requested).
Employee 2 Works shift 1 (requested).
Employee 4 Works shift 2 (requested).

Day 2
Employee 2 Works shift 2 (not requested).
Employee 3 Works shift 0 (requested).
Employee 4 Works shift 1 (requested).

Day 3
Employee 1 Works shift 1 (requested).
Employee 2 Works shift 0 (requested).
Employee 3 Works shift 2 (not requested).

Day 4
Employee 0 Works shift 2 (requested).
Employee 1 Works shift 1 (not requested).
Employee 4 Works shift 0 (requested).

Day 5
Employee 0 Works shift 2 (not requested).
Employee 3 Works shift 0 (requested).
Employee 4 Works shift 1 (requested).

Day 6
Employee 0 Works shift 0 (not requested).
Employee 1 Works shift 2 (requested).
Employee 3 Works shift 1 (not requested).



In [12]:
print('Statistics')
print('  - Number of shift requests met = %i' % solver.ObjectiveValue(),
      '(out of', num_employees * min_shifts_per_emp, ')')
print('  - Wall time       : %f s' % solver.WallTime())

Statistics
  - Number of shift requests met = 13 (out of 20 )
  - Wall time       : 0.013407 s
